In [14]:
import pandas as pd
import xarray as xr
import numpy as np
import copy
import math
import random
import warnings
import matplotlib.pyplot as plt
import os
from deap import base
from deap import creator
from deap import tools
import calliope
from calliope.exceptions import ModelWarning

calliope.set_log_verbosity(verbosity='INFO', include_solver_output=True, capture_warnings=True)
#warnings.filterwarnings("ignore", category=ModelWarning)

In [15]:
model = calliope.Model('Model/model.yaml', scenario='2050_eff,no_old_techs') 

[2025-01-16 17:33:03] INFO     Model: initialising
[2025-01-16 17:33:03] INFO     Applying the following overrides from scenario definition: ['2050_eff', 'no_old_techs'] 
[2025-01-16 17:33:04] INFO     Override applied to locations.CNOR.techs.demand_power.constraints.resource: file=regional_demand.csv -> file=demand_2050_eff.csv
Override applied to locations.CSUD.techs.demand_power.constraints.resource: file=regional_demand.csv -> file=demand_2050_eff.csv
Override applied to locations.NORD.techs.demand_power.constraints.resource: file=regional_demand.csv -> file=demand_2050_eff.csv
Override applied to locations.SARD.techs.demand_power.constraints.resource: file=regional_demand.csv -> file=demand_2050_eff.csv
Override applied to locations.SICI.techs.demand_power.constraints.resource: file=regional_demand.csv -> file=demand_2050_eff.csv
Override applied to locations.SUD.techs.demand_power.constraints.resource: file=regional_demand.csv -> file=demand_2050_eff.csv
`techs.ccgt.exists`:False

In [16]:
model.run()

df_total_cost = model.results.cost.to_series().dropna()
total_cost_optimal = df_total_cost.loc[~df_total_cost.index.map(str).str.contains('co2_emissions')].sum()

energy_cap_df = model.results.energy_cap.to_pandas()
filtered_energy_cap_df = energy_cap_df[
    ~energy_cap_df.index.str.contains("demand|transmission") &  # Exclude demand and transmission
    energy_cap_df.index.str.contains("battery|bio|hydro|phs|PV|wind")  # Include only the specified words
    energy_cap_df.index.str.contains("R1|R2|R")
]

[2025-01-16 17:33:10] INFO     Backend: starting model run
[2025-01-16 17:33:10] INFO     Loading sets
[2025-01-16 17:33:10] INFO     Loading parameters
[2025-01-16 17:33:10] INFO     constraints are loaded in the following order: ['capacity', 'costs', 'dispatch', 'energy_balance', 'network', 'policy', 'conversion', 'conversion_plus', 'group', 'export', 'milp']
[2025-01-16 17:33:10] INFO     creating capacity constraints
[2025-01-16 17:33:10] INFO     creating costs constraints
[2025-01-16 17:33:13] INFO     creating dispatch constraints
[2025-01-16 17:33:13] INFO     creating energy_balance constraints
[2025-01-16 17:33:13] INFO     creating network constraints
[2025-01-16 17:33:13] INFO     creating policy constraints
[2025-01-16 17:33:13] INFO     creating conversion constraints
[2025-01-16 17:33:13] INFO     creating conversion_plus constraints
[2025-01-16 17:33:13] INFO     creating group constraints
[2025-01-16 17:33:13] INFO     creating export constraints
[2025-01-16 17:33:13] 

In [17]:
model_14D = model

In [18]:
model_FULL = model

In [19]:
initial_capacities_FULL = filtered_energy_cap_df.values 

# if a value is lower than 13-5, set the value to 0. This is due to everything below 1e-5 being so low it it not interesting to include
initial_capacities_FULL[initial_capacities_FULL < 1e-5] = 0

# adjust when using more models
optimal_value = total_cost_optimal # adjust when using more models

In [20]:
print(filtered_energy_cap_df)

loc_techs
R14::biomass_wood          2500.000000
SICI::hydro_dam           15800.000000
SARD::hydro_ror           40000.000000
R6::wind                      0.000000
SARD::hydrogen_market      6928.384686
                             ...      
R14::wind_offshore         1227.274436
SARD::hydro_dam          183800.000000
R8::biofuel              264300.000000
R18::phs_new               5706.389845
R13::phs                 579200.000000
Length: 178, dtype: float64


In [8]:
updates = [
    {'tech': tech, 'loc': loc}
    for loc_tech in filtered_energy_cap_df.index
    for loc, tech in [loc_tech.split("::")]  # Split index by '::' to separate loc and tech
]

In [ ]:
print(updates)

In [10]:
input_params = model.backend.access_model_inputs()

# Access energy_cap_max and energy_cap_min for each technology
energy_cap_max = input_params['energy_cap_max']
energy_cap_min = input_params['energy_cap_min']

# Convert to DataFrame for filtering
energy_cap_max_df = energy_cap_max.to_dataframe()
energy_cap_min_df = energy_cap_min.to_dataframe()

# Filter out rows with 'demand' or 'free' in the index
energy_cap_max_filtered = energy_cap_max_df[~energy_cap_max_df.index.get_level_values('loc_techs').str.contains("demand|transmission")]
energy_cap_min_filtered = energy_cap_min_df[~energy_cap_min_df.index.get_level_values('loc_techs').str.contains("demand|transmission")]


# Create a dictionary of loc_tech to [min, max] bounds meaning that you will have the loc_tech and their corresponding min max capacity
low_up_mapping = {
    loc_tech: [
        energy_cap_min_filtered.loc[loc_tech, 'energy_cap_min'],
        energy_cap_max_filtered.loc[loc_tech, 'energy_cap_max']
    ]
    for loc_tech in energy_cap_max_filtered.index
}

# Structure the updates (previously done, which are the tech:, loc: pairs) so that it can eventually be used to put in the backend
updates_order = [f"{update['loc']}::{update['tech']}" for update in updates]

# Put it in such a structure that it can be used for the mutation function. This wants it in a list order
low_up_bound = [
    low_up_mapping[loc_tech] for loc_tech in updates_order
]

max_cap_value = 1e6

# Check for 'inf' in upper bounds and adjust if needed
for i, (low, up) in enumerate(low_up_bound):
    if up == float('inf'):
        low_up_bound[i][1] = max_cap_value


In [ ]:
print(low_up_bound)